# Vehicle Creation and Control Example

This notebook demonstrates how to spawn a vehicle and control it remotely using DCS Jupyter kernel.

## Prerequisites

1. **Place markers on the DCS map:**
   - Press `F10` to open the map
   - Right-click where you want to spawn the tank and select "Add mark"
   - Label it "SPAWN" (or any name you prefer)
   - Right-click somewhere else and add another marker labeled "TARGET"
   - Make sure both markers are on **land**, not water!

2. **Start the mission** and run this notebook

## Setup

First, let's set up our utility function for JSON output:

In [1]:
_ = net.lua2json

nil

## Step 1: Find Map Markers

First, let's find the spawn position from the map markers you placed:

In [2]:
-- Get all map markers
markers = world.getMarkPanels()

-- Find SPAWN marker
spawn_marker = nil
target_marker = nil

for i, marker in pairs(markers) do
    if string.upper(marker.text) == "SPAWN" then
        spawn_marker = marker
    elseif string.upper(marker.text) == "TARGET" then
        target_marker = marker
    end
end

if not spawn_marker then
    error("Please place a marker labeled 'SPAWN' on the map!")
end

if not target_marker then
    error("Please place a marker labeled 'TARGET' on the map!")
end

-- Display marker information
return _({
    spawn_pos = spawn_marker.pos,
    target_pos = target_marker.pos,
    spawn_text = spawn_marker.text,
    target_text = target_marker.text
})

{"target_pos":{"y":175.62407699109,"x":11503.03665637,"z":15321.948520692},"spawn_text":"SPAWN","target_text":"TARGET","spawn_pos":{"y":161.00019151761,"x":10646.219712053,"z":13325.847507556}}

## Step 2: Spawn Vehicle at Marker

Now let's spawn a tank at the SPAWN marker position:

In [3]:
-- Use spawn marker position
spawn_pos = spawn_marker.pos

-- Create a simple ground unit group
local tank_group = {
    name = "Remote_Tank_1",
    task = "Ground Nothing",
    units = {
        [1] = {
            name = "Tank_Unit_1",
            type = "M-1 Abrams",
            x = spawn_pos.x,
            y = spawn_pos.z,
            heading = 0,
            skill = "Average",
            playerCanDrive = true
        }
    }
}

-- Spawn the group
coalition.addGroup(country.id.USA, Group.Category.GROUND, tank_group)

-- Add green smoke at spawn position to mark it
trigger.action.smoke(spawn_pos, trigger.smokeColor.Green)

trigger.action.outText("Tank spawned at SPAWN marker (green smoke)", 10)

nil

## Step 3: Get Vehicle Reference

Find our spawned vehicle and get its controller:

In [4]:
-- Find the group we just created
tank_group = Group.getByName("Remote_Tank_1")
tank_unit = tank_group:getUnit(1)
tank_controller = tank_unit:getController()

-- Get current position
current_pos = tank_unit:getPoint()
return _(current_pos)

{"y":161.00120660078,"x":10646.219726566,"z":13325.847656255}

## Step 4: Move Tank to TARGET Marker

Let's move the tank to the TARGET marker position:

In [5]:
-- Use target marker position
target_pos = target_marker.pos

-- Create a simple move mission
move_mission = {
    id = 'Mission',
    params = {
        route = {
            points = {
                [1] = {
                    x = current_pos.x,
                    y = current_pos.z,
                    speed = 10,
                    action = "Off Road"
                },
                [2] = {
                    x = target_pos.x,
                    y = target_pos.z,
                    speed = 15,
                    action = "Off Road"
                }
            }
        }
    }
}

-- Send the mission to the tank
tank_controller:pushTask(move_mission)

-- Add red smoke at target position
trigger.action.smoke(target_pos, trigger.smokeColor.Red)

trigger.action.outText("Tank ordered to move to TARGET marker (red smoke)", 10)

nil

## Step 5: Monitor Vehicle Status

Check if the tank is moving and its current status:

In [6]:
-- Check if tank has tasks
has_task = tank_controller:hasTask()

-- Get current position
new_pos = tank_unit:getPoint()

-- Check if tank is active
is_active = tank_unit:isActive()

-- Get health status
life = tank_unit:getLife()

-- Calculate distance to target
distance_to_target = math.sqrt((new_pos.x - target_pos.x)^2 + (new_pos.z - target_pos.z)^2)

status = {
    has_task = has_task,
    position = new_pos,
    is_active = is_active,
    life = life,
    distance_to_target = distance_to_target
}

return _(status)

{"is_active":true,"life":32,"has_task":true,"position":{"y":161.11590769191,"x":10665.395771044,"z":13370.488194936},"distance_to_target":2123.6383062494}

## Step 6: Additional Visual Effects

Add a flare to mark the tank's current position:

In [7]:
-- Add flare at tank's current position
current_pos = tank_unit:getPoint()
trigger.action.signalFlare(current_pos, trigger.flareColor.Yellow, 0)

-- Show progress message
distance_remaining = math.sqrt((current_pos.x - target_pos.x)^2 + (current_pos.z - target_pos.z)^2)
trigger.action.outText("Tank position marked with YELLOW flare. Distance to target: " .. 
                       math.floor(distance_remaining) .. " meters", 10)

nil

## Step 7: Advanced Control

Stop the tank and give it a new task:

In [8]:
-- Stop current tasks
tank_controller:resetTask()

-- Set tank to hold position
hold_task = {
    id = 'Hold',
    params = {}
}

tank_controller:pushTask(hold_task)

trigger.action.outText("Tank ordered to hold position", 5)

nil

## Cleanup

Remove the spawned vehicle:

In [9]:
-- Destroy the group
tank_group:destroy()

trigger.action.outText("Tank removed", 5)

nil